In [1]:
import io
import h5py
from h5glance import H5Glance
import numpy as np
import os
import os.path as osp
import pandas as pd

from cfel_geom import CrystfelGeom  # Local file - move to EXtra-geom later

## Input data

In [2]:
# TODO: use a relative path to data generated in this folder
data_file = "/gpfs/exfel/data/user/juncheng/crystalProject/data/simulation/xstal/xstal100.h5"

In [3]:
f = h5py.File(data_file, 'r')

In [4]:
H5Glance(f)

Here's one frame from the simulation:

In [5]:
%matplotlib widget

In [6]:
geom = CrystfelGeom.from_crystfel_geom('tmp.geom')
frame = f['data/0000001/data'][:]
ax = geom.plot_data_fast(frame[np.newaxis], axis_units='m')

FigureCanvasNbAgg()

## Running CrystFEL

In [7]:
with open('test.lst', 'w') as f:
    f.write(f'{data_file}\n')

In [8]:
%%bash
source /usr/share/Modules/init/bash
module load exfel exfel_crystfel

indexamajig -i test.lst -g tmp.geom -o xstal.stream --peaks=zaef -p ./lysozyme-opt.cell

 - EXFEL modulepath enabled
Using version 1.10.4 of HDF5
Using version 1.17 of DirAx
Using XDS BUILT 20180126
Unrecognised field 'px'
Unrecognised field 'py'
Unrecognised field 'pix_width'
Unrecognised field 'd'
No indexing methods specified.  I will try to automatically detect the available methods.
To disable auto-detection of indexing methods, specify which methods to use with --indexing=<methods>.
Use --indexing=none to disable indexing and integration.
This is what I understood your unit cell to be:
tetragonal P, unique axis c, right handed.
a      b      c            alpha   beta  gamma
 78.92  78.92  37.88 A     90.00  90.00  90.00 deg
a =  7.892e-09  0.000e+00  0.000e+00 m
b =  4.832e-25  7.892e-09  0.000e+00 m
c =  2.319e-25  2.319e-25  3.788e-09 m
a* =  1.267e+08 -7.759e-09 -7.759e-09 m^-1 (modulus  1.267e+08 m^-1)
b* =  0.000e+00  1.267e+08 -7.759e-09 m^-1 (modulus  1.267e+08 m^-1)
c* =  0.000e+00  0.000e+00  2.640e+08 m^-1 (modulus  2.640e+08 m^-1)
alpha* =  90.00 deg, beta

## Identified peaks

In [9]:
def read_stream_table(fh, end_marker):
    buf = io.StringIO()
    for line in fh:
        if line.startswith(end_marker):
            buf.seek(0)
            return pd.read_fwf(buf)
        
        buf.write(line)

In [10]:
stream_chunks = {}

with open('xstal.stream', 'r') as f:
    in_peaks_table = False
    chunk_data = {}
    for line in f:
        if '-- Begin chunk --' in line:
            chunk_data = {}
        elif '-- End chunk --' in line:
            if 'event' in chunk_data:
                stream_chunks[chunk_data['event']] = chunk_data
        
        elif line.startswith('Event:'):
            chunk_data['event'] = line.split(':', 1)[1].strip()
        
        elif line.startswith('Peaks from peak search'):
            chunk_data['peaks'] = read_stream_table(f, 'End of peak list')

In [11]:
peak_tbl = stream_chunks['0000001//']['peaks']

In [12]:
module_no = np.zeros(len(peak_tbl), dtype=np.uint32)
peak_coords = geom.data_coords_to_positions(module_no, peak_tbl['ss/px'], peak_tbl['fs/px'])
peak_x = peak_coords[:, 0]
peak_y = peak_coords[:, 1]

In [13]:
ax = geom.plot_data_fast(frame[np.newaxis], axis_units='m')
ax.scatter(peak_x, peak_y, s=30, marker='o', facecolor='none', edgecolor='0.5')

FigureCanvasNbAgg()

## Examine unit cell parameters

In [14]:
from CrystFEL_Jupyter_utilities.GUI_tools import CellExplorer

In [15]:
ce = CellExplorer('xstal.stream')

FigureCanvasNbAgg()